In [4]:
import numpy as np
import tensorflow as tf
from scipy import ndimage
import math
import cv2
import os
# create an array where we can store our 2 pictures
images = np.zeros((8,784))
# and the correct values
correct_vals = np.zeros((8,46))

# we want to test our images which you saw at the top of this page
i = 0
labels = [14,32,21,24,31,32,22,21]

for letter in ["W","E1","L1","O","V","E2","M","L2"]:
    # read the image
    gray = cv2.imread(letter+"mendola.png", 0)
    #print(gray)
    #invert image
    #gray = cv2.bitwise_not(gray)

    _, gray = cv2.threshold(255-gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # resize the images
    gray = cv2.resize(gray,(28, 28))
    # save the processed images
    while np.sum(gray[0]) == 0:
        gray = gray[1:]

    while np.sum(gray[:,0]) == 0:
        gray = np.delete(gray,0,1)

    while np.sum(gray[-1]) == 0:
        gray = gray[:-1]

    while np.sum(gray[:,-1]) == 0:
        gray = np.delete(gray,-1,1)
    
    rows,cols = gray.shape

    #gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant') 
    if rows > cols:
        factor = 20.0/rows
        rows = 20
        cols = int(round(cols*factor))
        gray = cv2.resize(gray, (cols,rows))
    else:
        factor = 20.0/cols
        cols = 20
        rows = int(round(rows*factor))
        gray = cv2.resize(gray, (cols, rows))
    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    cy,cx = ndimage.measurements.center_of_mass(gray)
    rows,cols = gray.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')
    """
    all images in the training set have an range from 0-1
    and not from 0-255 so we divide our flatten images
    (a one dimensional vector with our 784 pixels)
    to use the same 0-1 based range
    """
    flatten = gray.flatten() / 255.0
    """
    we need to store the flatten image and generate
    the correct_vals array
    correct_val for the first digit (9) would be
    [0,0,0,0,0,0,0,0,0,1]
    """
    cv2.imwrite(letter+"mendolaPRO.png", gray)
    print (gray)
    print (flatten)
    #print(len(images[i]))
    #print(len(flatten))
    images[i] = flatten
    correct_val = np.zeros((46))
    correct_val[labels[i]] = 1
    correct_vals[i] = correct_val
    i += 1

print (images)
"""
the prediction will be an array with four values,
which show the predicted number
prediction = tf.argmax(y,1)

we want to run the prediction and the accuracy function
using our generated arrays (images and correct_vals)
print sess.run(prediction, feed_dict={x: images, y_: correct_vals})
print sess.run(accuracy, feed_dict={x: images, y_: correct_vals})
"""

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  12   4   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 182 103  35   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 137 149  86   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  13 158 128   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 170 142   0   0   0   0   0   0   0   

'\nthe prediction will be an array with four values,\nwhich show the predicted number\nprediction = tf.argmax(y,1)\n\nwe want to run the prediction and the accuracy function\nusing our generated arrays (images and correct_vals)\nprint sess.run(prediction, feed_dict={x: images, y_: correct_vals})\nprint sess.run(accuracy, feed_dict={x: images, y_: correct_vals})\n'